# Scraping Welcome to The Jungle

## Import des librairies

In [4]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import locale
import pandas as pd
from selenium import webdriver
import time
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver=webdriver.Chrome(options=options)

## Collecte des données d'une offre

In [5]:
def get_base_url(category, location ="FR"):
    url=("https://www.welcometothejungle.com/fr/jobs"
         f"?query={category}"
        f"&refinementList%5Boffices.country_code%5D%5B%5D={location}")
    return url

In [6]:
get_base_url('data')

'https://www.welcometothejungle.com/fr/jobs?query=data&refinementList%5Boffices.country_code%5D%5B%5D=FR'

In [7]:
url = "https://www.welcometothejungle.com/fr/companies/lita-co/jobs/stagiaire-communication-et-reseaux-sociaux_paris?q=052d7b4586a4ba590c6b4be0deb0b4f3&o=a10b0b85-3607-46d0-b816-0954fbf473fe&p=true"
response = requests.get(url)

In [8]:
response

<Response [200]>

In [9]:
response.reason

'OK'

In [10]:
response.text

'<!DOCTYPE html><html lang="fr"><head><script type="text/javascript">window.env = {"ALERTS_API_HOST":"alerts.welcometothejungle.com","ALGOLIA_API_KEY_CLIENT":"4bd8f6215d0cc52b26430765769e65a0","ALGOLIA_APPLICATION_ID":"CSEKHVMS53","ALGOLIA_APPLICATIONS_INDEX":"wk_cms_private_applications_production","ALGOLIA_ARTICLES_INDEX":"wk_cms_articles_production","ALGOLIA_JOBS_INDEX_PREFIX":"wttj_jobs_production","ALGOLIA_ORGANIZATIONS_INDEX":"wk_cms_organizations_production","CDN_HOST":"https://cdn.welcometothejungle.com","CDN_IMAGES_HOST":"https://cdn-images.welcometothejungle.com","EBA_API_HOST":"employerbrand.welcometothejungle.com","EXPANSION_GLOBAL_HOST":"https://global.welcometothejungle.com","EXPANSION_US_HOST":"https://us.welcometothejungle.com","EXPANSION_UK_HOST":"https://uk.welcometothejungle.com","GOOGLE_CLIENT_ID":"405382547658-470vuq9vapq41apfhvlp5p4topbcic2t.apps.googleusercontent.com","GTM_CONTAINER_ID":"GTM-WZZZJQ3","HERE_API_KEY":"3YHjVgEYjuwUatQAtD-wTX8lmNXEsULPzC8m59VMGDw","H

In [11]:
soup = BeautifulSoup(response.text, 'html.parser')

In [12]:
company=soup.find('div',class_='sc-bXCLTC dPVkkc').find('span').text.strip()
print(company)

LITA.co


In [13]:
job_title= soup.find('h2').text.strip()
print(job_title)

Chargé.e Communication et Réseaux Sociaux (stage)


In [14]:
contract_tag= soup.find('div',class_='sc-fKWMtX lpfkog')
print(contract_tag)
if contract_tag:
    contract=contract_tag.get_text(separator=" ").strip()
    print(contract)

<div class="sc-fKWMtX lpfkog" variant="default" w="fit-content"><i class="sc-kbdlSk gjPEnb wui-icon-font" name="contract"></i>Stage<span>(5 à 6 mois)</span></div>
Stage (5 à 6 mois)


In [15]:
lieu_tag=soup.find('span',class_='q7vo0q-0 dhOyPm')
print(lieu_tag)
lieu = lieu_tag.text.strip()
print(lieu)

<span class="q7vo0q-0 dhOyPm">Paris</span>
Paris


In [16]:
resume_tag=soup.find_all('div',class_='sc-fKWMtX lpfkog')
print(resume_tag)
results={}
for info in resume_tag:
    name=info.find("i")["name"] if info.find("i") else "unknown"
    text= info.get_text(separator=" ").strip()
    if "Salaire :" in text:
            text=text.replace("Salaire : ","")
    if "Éducation :" in text:
            text=text.replace("Éducation : ","")
    results[name]=text
print(results)

[<div class="sc-fKWMtX lpfkog" variant="default" w="fit-content"><i class="sc-kbdlSk gjPEnb wui-icon-font" name="contract"></i>Stage<span>(5 à 6 mois)</span></div>, <div class="sc-fKWMtX lpfkog" variant="default" w="fit-content"><i class="sc-kbdlSk kiqvoY wui-icon-font" name="location"></i><span class="q7vo0q-2 hKloXu"><span class="q7vo0q-0 dhOyPm">Paris</span></span></div>, <div class="sc-fKWMtX lpfkog" variant="default" w="fit-content"><i class="sc-kbdlSk eKXXcm wui-icon-font" name="salary"></i><span class="sc-bXCLTC kJcLKT">Salaire : </span>Non spécifié</div>, <div class="sc-fKWMtX lpfkog" variant="default" w="fit-content"><i class="sc-kbdlSk bwoqTp wui-icon-font" name="remote"></i><span>Télétravail occasionnel</span><span class="sc-bXCLTC iGeUCm"><i class="sc-kbdlSk feznw wui-icon-font" name="information_outline"></i></span><span hidden="" id=":Rbdj2an3a:" style="position:fixed"></span></div>, <div class="sc-fKWMtX lpfkog" variant="default" w="fit-content"><i class="sc-kbdlSk lQaKo

In [17]:
time_tag = soup.find("time")
#date_announcement = time_tag.find("span").text
#print(date_announcement)
date= time_tag["datetime"]
date_obj= datetime.strptime(date.replace("Z",""),"%Y-%m-%dT%H:%M:%S")
rdate= date_obj.strftime("%A %d %B %Y, %H:%M %p")
print(rdate)

Thursday 28 November 2024, 10:48 AM


In [18]:
try:
    description = soup.find("div",class_="sc-1tacsq-1 bHGOjt").text
except Exception :
    description =""
print(description)

Le Pôle Marketing & Communication travaille sur le développement de la notoriété de la marque LITA auprès du grand public et des entrepreneurs, ainsi que la sensibilisation sur les enjeux d’une épargne plus durable ; et participe directement à la réussite des levées de fonds participatives proposées sur la plateforme.En tant que stagiaire communication et réseaux sociaux, tu auras pour mission principale d’assister l’équipe communication. Tu contribueras à la création, à la mise en œuvre et à l’optimisation des stratégies de contenu et de communication pour le blog, les newsletters et les réseaux sociaux de la marque.Tu pourras aussi être amené·e à travailler en collaboration avec d’autres pôles, sur des projets spécifiques.Au quotidien, tes missions seront les suivantes :1- CommunicationPréparation de contenus éditoriaux : recherche de thématiques, notes de lecture et synthèseRédaction de contenus : communiqués de presse, articles de blog (brèves, articles SEO)Relecture approfondie (e

## Création d'une fonction qui récupère toutes les informations sur une offre

In [19]:
def get_info(link):
    response = requests.get(link)
    response.text
    soup = BeautifulSoup(response.text, 'html.parser')
    offre={}
    
    #comp_tag= soup.find('div',class_='sc-bXCLTC dPVkkc')
    try:
        offre['company']=soup.find('a',class_='sc-eHsDsR gCOjps').find('span').text.strip()
    except Exception :
        offre['company'] =""
    try:
        offre['job_title']= soup.find('h2').text.strip()
    except Exception :
        offre['job_title'] =""
    try:    
        time_tag = soup.find("time")
        date= time_tag["datetime"]
        date_obj= datetime.strptime(date.replace("Z",""),"%Y-%m-%dT%H:%M:%S")
        rdate= date_obj.strftime("%A %d %B %Y, %H:%M %p")
        offre["date"] = rdate
    except Exception :
        offre['date'] =""
    resume_tag=soup.find_all('div',class_='sc-fKWMtX lpfkog')
    for info in resume_tag:
        name=info.find("i")["name"] if info.find("i") else "unknown"
        text= info.get_text(separator=" ").strip()
        if "Salaire : " in text:
            text=text.replace("Salaire : ","")
        if "Éducation :" in text:
            text=text.replace("Éducation : ","")
        if "Début : " in text:
            text=text.replace("Début : ","")
        if "Expérience : " in text:
            text=text.replace("Expérience : ","")
        
        offre[name]=text
    
    try:
        description_ = soup.find("div",class_="sc-1tacsq-1 bHGOjt").text
    except Exception :
        description_ =""
    offre['description']=description_ 
    
    offre['lien']=link
    return offre

In [20]:
get_info(url)

{'company': 'LITA.co',
 'job_title': 'Chargé.e Communication et Réseaux Sociaux (stage)',
 'date': 'Thursday 28 November 2024, 10:48 AM',
 'contract': 'Stage (5 à 6 mois)',
 'location': 'Paris',
 'salary': ' Non spécifié',
 'remote': 'Télétravail occasionnel',
 'education_level': ' Bac +5 / Master',
 'description': 'Le Pôle Marketing & Communication travaille sur le développement de la notoriété de la marque LITA auprès du grand public et des entrepreneurs, ainsi que la sensibilisation sur les enjeux d’une épargne plus durable\u202f; et participe directement à la réussite des levées de fonds participatives proposées sur la plateforme.En tant que stagiaire communication et réseaux sociaux, tu auras pour mission principale d’assister l’équipe communication. Tu contribueras à la création, à la mise en œuvre et à l’optimisation des stratégies de contenu et de communication pour le blog, les newsletters et les réseaux sociaux de la marque.Tu pourras aussi être amené·e à travailler en collab